In [117]:
import dill 
import torch 
import torch.nn as nn
import numpy as np
from four_room.shortest_path import find_all_action_values, find_all_shortest_paths
from four_room.env import FourRoomsEnv
from four_room.wrappers import gym_wrapper
import numpy as np
import imageio
import gymnasium as gym
from four_room.utils import obs_to_state
from four_room.arch import CNN
from four_room.constants import state_to_q
from rnd_exploration.dataset import State
from tqdm import tqdm
import matplotlib.pyplot as plt
import random


import warnings
warnings.filterwarnings(action='once')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [118]:
gym.register('MiniGrid-FourRooms-v1', FourRoomsEnv)
size = 19
with open('configs/train.pl', 'rb') as file:
    train_config = dill.load(file)

with open('configs/test_reachable.pl', 'rb') as file:
    test_config = dill.load(file)

with open('configs/validation_unreachable.pl', 'rb') as file:
    val_config = dill.load(file)

c:\Users\garga\Documents\Uni\four_room\venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment MiniGrid-FourRooms-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [119]:
env = gym_wrapper(gym.make(
        'MiniGrid-FourRooms-v1', 
        agent_pos= train_config['agent positions'],
        goal_pos = train_config['goal positions'],
        doors_pos = train_config['topologies'],
        agent_dir = train_config['agent directions'],
        size=size, 
        max_steps=1200, 
    ),
    original_obs=True
)

In [ ]:
"""combos
next == prev -> forward 
next | prev
2 - 3 < 0 -> left
1 - 2 < 0 -> left
0 - 1 < 0 -> left
3 - 0 < 0 -> left

1 - 0 > 0 -> right
2 - 1 > 0 -> right
3 - 2 > 0 -> right
0 - 3 > 0 -> right

left = 0 
right = 1
forward = 2
"""


In [120]:
def compute_actions(path: list):
    actions = []
    action_dict = {
        (2, 3) : 0,
        (1, 2) : 0, 
        (0, 1) : 0, 
        (3, 0) : 0, 
        (1, 0) : 1,
        (2, 1) : 1, 
        (3, 2) : 1,
        (0, 3) : 1,
    }
    for prev, next in zip(path[:-1], path[1:]):
        if prev[2] == next[2]:
            actions.append(2)
        else:
            actions.append(action_dict[(int(next[2]), int(prev[2]))])
        
            
    return actions

In [121]:
obs, _ = env.reset()

In [122]:
max_k = len(env.get_wrapper_attr('valid_pos'))
k = np.random.randint(low=0, high=max_k)
aux_pos = env.get_wrapper_attr('valid_pos')[k]

In [123]:
state = obs_to_state(obs)
q = find_all_shortest_paths(state[:2], state[2], aux_pos, state[5:], size)

In [124]:
q[0]

[(np.int64(12), np.int64(6), np.uint8(0)),
 (13, 6, 0),
 (14, 6, 0),
 (15, 6, 0),
 (15, 6, 1),
 (15, 7, 1),
 (15, 8, 1),
 (np.int64(15), 9, 1),
 (np.int64(15), 10, 1),
 (15, 11, 1),
 (15, 11, 0),
 (16, 11, 0),
 (17, 11, 0)]

In [112]:
compute_actions(q[0])

[0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2]

In [ ]:
aux_reached = 0

for i in range(len(train_config['topologies']) * 3):

    obs, _ = env.reset() 
    done = False 
    max_k = len(env.get_wrapper_attr('valid_pos'))
    k = np.random.randint(low=0, high=max_k)
    aux_pos = env.get_wrapper_attr('valid_pos')[k]
    
    state = obs_to_state(obs)
    goal_pos = state[3:5]
    agent_pos = state[:2]
    reached = False
    target_pos = aux_pos
    
    paths = find_all_shortest_paths(state[:2], state[2], target_pos, state[5:], size)
    actions = compute_actions(paths[0])
    print(f'========{i}==========')
        
    
    while not done:
        state = obs_to_state(obs)
        agent_pos = env.get_wrapper_attr('agent_pos')
        
        print('------')
        print('aux pos', aux_pos)
        print('target_pos', target_pos)
        print('agent_pos', agent_pos, state[2])
        print('goal_pos', goal_pos)
        print('actions', actions)
        
        if np.array_equal(target_pos, aux_pos) and not np.array_equal(agent_pos, aux_pos):
            action = actions.pop(0)
        else:
            state_obj = State(state=obs)
            q = state_to_q[state_obj]
            action = q.argmax()
        
        obs, reward, terminated, truncated, _ = env.step(action)        
        agent_pos_after = env.get_wrapper_attr('agent_pos')
        
        if (np.array_equal(agent_pos_after, aux_pos) or np.array_equal(agent_pos, aux_pos)) and not reached:
            reached = True
            target_pos = goal_pos
            aux_reached += 1
        
        # print(f'Context is {i+1} | {agent_pos_after} | {aux_pos} | target reached: {aux_reached} | {target_pos}', end='\r')
        
        done = terminated or truncated
        

========0==========
------
aux pos (np.int64(10), np.int64(11))
target_pos (np.int64(10), np.int64(11))
agent_pos (10, 11) 0
goal_pos (np.int64(1), np.int64(12))
actions []
------
aux pos (np.int64(10), np.int64(11))
target_pos (np.int64(1), np.int64(12))
agent_pos (10, 11) 1
goal_pos (np.int64(1), np.int64(12))
actions []
------
aux pos (np.int64(10), np.int64(11))
target_pos (np.int64(1), np.int64(12))
agent_pos (np.int64(10), np.int64(12)) 1
goal_pos (np.int64(1), np.int64(12))
actions []
------
aux pos (np.int64(10), np.int64(11))
target_pos (np.int64(1), np.int64(12))
agent_pos (np.int64(10), np.int64(13)) 1
goal_pos (np.int64(1), np.int64(12))
actions []
------
aux pos (np.int64(10), np.int64(11))
target_pos (np.int64(1), np.int64(12))
agent_pos (np.int64(10), np.int64(14)) 1
goal_pos (np.int64(1), np.int64(12))
actions []
------
aux pos (np.int64(10), np.int64(11))
target_pos (np.int64(1), np.int64(12))
agent_pos (np.int64(10), np.int64(15)) 1
goal_pos (np.int64(1), np.int64(12)

In [9]:
from collections import defaultdict

class Dataset:
    """Basic dataset class. Makes it easier to pickle the file. 
    """
    
    def __init__(self):
        self.X = []
        self.Y = []
        self.states = []
        
    def add(self, x, y, state):
        """Add a data sample to the dataset"""
        self.X.append(x)
        self.Y.append(y)
        self.states.append(state)
        
    def sort(self):
        """Returns a dictionary which for each q-vector returns the  
        corresponding states. 
        """             
        binned_dict = defaultdict(list)
        
        X = np.array(self.X)
        Y = np.array(self.Y)
        
        for q in np.unique(Y, axis=0):
            mask = np.all(Y == q, axis=1)
            binned_dict[tuple(q)] = X[mask]
        return binned_dict
    
    def __getitem__(self, index):
        return (self.X[index], self.Y[index])
    
    def __len__(self):
        return len(self.X)
    

In [11]:
from hashlib import sha1

class State: 
    
    def __init__(
        self,
        state: np.ndarray
    ):
        self.state = state
        
    def __eq__(self, other):
        return bool(np.all(self.state == other.state))
    
    def __hash__(self):
        state_hash = int(sha1(self.state.flatten()).hexdigest(), 16)
        return hash(state_hash)
        
class Transition:
    
    def __init__(
        self, 
        state: np.ndarray, 
        q_value: np.ndarray
    ):  
        self.state = state
        self.q_value = q_value
        
    def __eq__(self, other):
        return bool(np.all(self.state == other.state) and np.all(self.q_value == other.q_value))
    
    def __hash__(self):
        state_hash = int(sha1(self.state.flatten()).hexdigest(), 16)
        q_value_hash = int(sha1(self.q_value.flatten()).hexdigest(), 16)
        return hash((state_hash, q_value_hash))
    
class Trajectory: 
    
    def __init__(
        self
    ):
        self.transitions = []
        self.unique_transitions = set([])
    
    def add(self, transition: Transition):
        self.transitions.append(transition)
        self.unique_transitions.add(transition)
    
    def __eq__(self, other):
        if len(self.transitions) != len(other.transitions):
            return False
        return all(t1 == t2 for t1, t2 in zip(self.transitions, other.transitions))
    
    def __hash__(self):
        t_matrix_state = np.array([t.state for t in self.transitions]).flatten() 
        t_matrix_q = np.array([t.state for t in self.transitions]).flatten() 
        state_hash = int(sha1(t_matrix_state).hexdigest(), 16)
        q_value_hash = int(sha1(t_matrix_q).hexdigest(), 16)
        return hash((state_hash, q_value_hash))

    def uniqueness(self, other):
        intersection = self.unique_transitions.intersection(other.unique_transitions)
        return len(intersection) / len(self.transitions) # compares how unique is our trajectory compared to other
    
    def __iter__(self):
        return iter([(t.state, t.q_value) for t in self.transitions])
    
class ExploreGoDataset(Dataset):
    
    def __init__(self):
        super().__init__()
        
        self.trajectories = []
        self.unique_trajectories = set([])
        self.trans = []
        self.unique_trans = set([])
        self.current_traj = Trajectory()
        
    def wrap_trajectory(self):
        self.trajectories.append(self.current_traj)
        self.unique_trajectories.add(self.current_traj)
        for state, q in self.current_traj:
            self.add(state, q, None) # not storing the state
        self.current_traj = Trajectory()
        
    def reset(self): 
        self.current_traj = Trajectory()
    
    def add_traj(self, obs: np.ndarray, q_value: np.ndarray, state: np.ndarray):
        transition = Transition(obs, q_value)
        self.current_traj.add(transition)
        self.unique_trans.add(transition)
        
    def add_trans(self, obs: np.ndarray, q_value: np.ndarray):
        transition = Transition(obs, q_value)
        self.trans.append(transition)
        self.unique_trans.add(transition)
        
    def traj_uniqueness(self, trajectory: Trajectory): 
        unique = 0
        for traj in self.unique_trajectories:
            unique = max(unique, trajectory.uniqueness(traj))
        return unique  
    
    @property
    def ratio_unique_trans(self):
        return len(self.unique_trans) / len(self.trans)

In [20]:
def create_explorego_dataset(dataset_size, save_dir, K=100, render=False):
    if render:
        env = gym_wrapper(gym.make(
                'MiniGrid-FourRooms-v1', 
                agent_pos= train_config['agent positions'],
                goal_pos = train_config['goal positions'],
                doors_pos = train_config['topologies'],
                agent_dir = train_config['agent directions'],
                size=size, 
                render_mode="rgb_array",
            ),
            original_obs=True
        )
    else:
        env = gym_wrapper(gym.make(
                'MiniGrid-FourRooms-v1', 
                agent_pos= train_config['agent positions'],
                goal_pos = train_config['goal positions'],
                doors_pos = train_config['topologies'],
                agent_dir = train_config['agent directions'],
                size=size, 
            ),
            original_obs=True
        )
    
    explorego = ExploreGoDataset()
    imgs = []
    
    ep_highlight_mask = np.zeros((len(train_config['agent positions']), 
                                      env.get_wrapper_attr('width'), env.get_wrapper_attr('height')), dtype=bool)
    ep_colors = np.empty_like(ep_highlight_mask, dtype=object)

    while len(explorego) <= dataset_size:
        
        obs, _ = env.reset()
        done = False
        
        # emulate the pure exploration of explorego
        if np.random.random() < 0.6:
            max_k = len(env.get_wrapper_attr('valid_pos'))
            k = np.random.randint(low=0, high=max_k)
            env.get_wrapper_attr('move_valid_pos')(k)
        current_context = env.unwrapped.context
        
        # done_before = False
        # explore_steps = np.random.randint(low=0, high=K)
        # for _ in range(explore_steps):
        #     agent_pos = env.get_wrapper_attr('agent_pos')
            
        #     ep_highlight_mask[current_context, agent_pos[0], agent_pos[1]] = True
        #     ep_colors[current_context, agent_pos[0], agent_pos[1]] = (255, 128, 0)
        #     action = env.action_space.sample()
        #     obs, _, terminated, truncated, _ = env.step(action)
            
        #     done_before = terminated or truncated
        #     if render: imgs.append(env.render(highlight_mask=ep_highlight_mask[current_context], 
        #                                       colors=ep_colors[current_context]))
        #     if done_before:
        #         break
        
        # done = done_before
        
        # find optimal trajectory
        while not done:
            agent_pos = env.get_wrapper_attr('agent_pos')
            ep_highlight_mask[current_context, agent_pos[0], agent_pos[1]] = True
            ep_colors[current_context, agent_pos[0], agent_pos[1]] = (0, 204, 0)
            
            state = obs_to_state(obs)
            q = find_all_action_values(state[:2], state[2], state[3:5], state[5:], 0.99, size)
            q = np.array(q)
            explorego.add_trans(np.array(obs), q)
            explorego.add(np.array(obs), q, np.array(state))
            obs_prime, _, terminated, truncated, _ = env.step(q.argmax())
            obs = obs_prime
            done = terminated or truncated
            if render and len(explorego) >= dataset_size - 10000: imgs.append(env.render(highlight_mask=ep_highlight_mask[current_context], 
                                              colors=ep_colors[current_context]))
            
        print(f'Current size of dataset: {len(explorego):08d} | Current Context {current_context:04d} | Current Uniqueness {explorego.ratio_unique_trans:.4f}', end='\r')

    # save the obj       
    with open(f'action_values/{save_dir}.pl', 'wb') as file:
        dill.dump(explorego, file)
        
    return explorego, imgs

In [9]:
def create_avoid_dataset(dataset_size, save_dir, min_overlap=0.3, K=30, render=False):
    # storing avoid dataset
    overlapdataset = ExploreGoDataset()
    if render:
        env = gym_wrapper(gym.make(
                'MiniGrid-FourRooms-v1', 
                agent_pos= train_config['agent positions'],
                goal_pos = train_config['goal positions'],
                doors_pos = train_config['topologies'],
                agent_dir = train_config['agent directions'],
                size=size, 
                render_mode="rgb_array",
            ),
            original_obs=True
        )
    else:
        env = gym_wrapper(gym.make(
                'MiniGrid-FourRooms-v1', 
                agent_pos= train_config['agent positions'],
                goal_pos = train_config['goal positions'],
                doors_pos = train_config['topologies'],
                agent_dir = train_config['agent directions'],
                size=size, 
            ),
            original_obs=True
        )
    try:
        
    
        explorego = ExploreGoDataset()
        imgs = []
        
        ep_highlight_mask = np.zeros((len(train_config['agent positions']), 
                                        env.get_wrapper_attr('width'), env.get_wrapper_attr('height')), dtype=bool)
        ep_colors = np.empty_like(ep_highlight_mask, dtype=object)

        while len(explorego) <= dataset_size:
            
            obs, _ = env.reset()
            done = False
            
            # emulate the pure exploration of explorego
            max_k = len(env.get_wrapper_attr('valid_pos'))
            k = np.random.randint(low=0, high=max_k)
            env.unwrapped.move_valid_pos(k)
        
            current_context = env.unwrapped.context
            
            
            # find optimal trajectory
            past_pos = []
            while not done:
                agent_pos = env.get_wrapper_attr('agent_pos')

                state = obs_to_state(obs)
                q = find_all_action_values(state[:2], state[2], state[3:5], state[5:], 0.99, size)
                q = np.array(q)
                
                transition = Transition(np.array(obs), q)
                if transition not in explorego.unique_trans:
                    explorego.add_trans(np.array(obs), q)
                    explorego.add(np.array(obs), q, np.array(state))
                    if render: 
                        ep_colors[current_context, agent_pos[0], agent_pos[1]] = (0, 0, 255)
                        ep_highlight_mask[current_context, agent_pos[0], agent_pos[1]] = True
                        past_pos.append(agent_pos)
                else:
                    if 1-explorego.ratio_unique_trans <= min_overlap:
                        explorego.add_trans(np.array(obs), q)
                        explorego.add(np.array(obs), q, np.array(state))
                        if render: 
                            ep_colors[current_context, agent_pos[0], agent_pos[1]] = (0, 0, 255)
                            ep_highlight_mask[current_context, agent_pos[0], agent_pos[1]] = True
                            past_pos.append(agent_pos)
                    else:
                        pass
                
                obs_prime, _, terminated, truncated, _ = env.step(q.argmax())
                obs = obs_prime
                done = terminated or truncated
                if render and len(explorego) >= dataset_size - 1000: imgs.append(env.render(highlight_mask=ep_highlight_mask[current_context], 
                                                colors=ep_colors[current_context]))
            if render:
                for pos in past_pos:
                    ep_colors[current_context, pos[0], pos[1]] = (51, 0, 102)

            print(f'Current size of dataset: {len(explorego):08d} | Current Context {current_context} | Current Uniqueness {explorego.ratio_unique_trans:.4f}', end='\r')

    except KeyboardInterrupt:
        with open(f'action_values/{save_dir}.pl', 'wb') as file:
            dill.dump(explorego, file)
        
    # save the obj       
    with open(f'action_values/{save_dir}.pl', 'wb') as file:
        dill.dump(explorego, file)
        
    print(explorego.ratio_unique_trans)
        
    return explorego, imgs

In [21]:
create_explorego_dataset(25000, 'explorego_25k') # NOTE: i accidentally overwrote this so its now 80k
create_explorego_dataset(50000, 'explorego_50k')
create_explorego_dataset(80000, 'explorego_80k')

c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


(<__main__.ExploreGoDataset at 0x14ee1c43450>, [])

In [8]:
create_avoid_dataset(25000, 'avoid_25k_04', min_overlap=0.4)
create_avoid_dataset(50000, 'avoid_50k_04', min_overlap=0.4)
create_avoid_dataset(80000, 'avoid_80k_04', min_overlap=0.4)
create_avoid_dataset(25000, 'avoid_25k_02', min_overlap=0.2)
create_avoid_dataset(50000, 'avoid_50k_02', min_overlap=0.2)
create_avoid_dataset(80000, 'avoid_80k_02', min_overlap=0.2)

c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


0.5999920006399488taset: 00025002 | Current Context 175 | Current Uniqueness 0.6000


c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


0.6000079998400032taset: 00050001 | Current Context 12 | Current Uniqueness 0.60000


c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


...

c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


0.7999840003199936taset: 00050001 | Current Context 57 | Current Uniqueness 0.80000


c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


0.7999900001249984taset: 00080001 | Current Context 82 | Current Uniqueness 0.80000


(<__main__.ExploreGoDataset at 0x227e7fb8c50>, [])

In [11]:
state_to_q = {}

for i in range(len(train_config['topologies'])):
    pairs_explored = []

    obs, _ = env.reset() 
    done = False 
    
    
    for idx in range(len(env.unwrapped.valid_pos)):
        env.get_wrapper_attr('move_valid_pos')(idx) 
        agent_pos = env.get_wrapper_attr('valid_pos')[idx]
        
        for _ in range(4): 
            obs, _, _, _, _ = env.step(1)
            state = obs_to_state(obs)
            q = find_all_action_values(state[:2], state[2], state[3:5], state[5:], 0.99, size)
            ind = State(obs)
            state_to_q[ind] = q
        
        print(f'Context is {i+1} | {agent_pos}', end='\r')
    
print(len(state_to_q))
with open('action_values/state_to_q.pl', 'wb') as file:
    dill.dump(state_to_q, file)


c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):


207200t is 200 | (9, 10))


In [30]:
with open('configs/state_to_q.pl', 'rb') as file:
    dict = dill.load(file)

In [31]:
obs, _ = env.reset()

In [32]:
dict[State(state=obs)]

[np.float64(0.8345137614500875),
 np.float64(0.8514577710948755),
 np.float64(0.8345137614500875)]